# X. Read Me

The following notebook was created for a final project for CS 6741. The goal was to determine low-cost methods for increasing 2-class toxicity classification using the huggingface jigsaw-toxicity-pred dataset. 

#1. Set-up 

## 1a. Install python packages

In [ ]:
# Install the libraries necessary for HuggingFace datasets and transformers
!pip install datasets
!pip install transformers
#!pip install sacremoses
#!pip install tokenizers
#!pip install sentencepiece

     |████████████████████████████████| 225kB 13.2MB/s 
     |████████████████████████████████| 112kB 18.4MB/s 
     |████████████████████████████████| 245kB 20.2MB/s 
     |████████████████████████████████| 2.3MB 9.6MB/s 
     |████████████████████████████████| 3.3MB 41.0MB/s 
     |████████████████████████████████| 901kB 43.4MB/s 


In [ ]:
#importing packages
from google.colab import drive

import datasets
from datasets import Dataset, load_dataset, concatenate_datasets
from torch.utils.data import Dataset, DataLoader

import torch

import transformers as transformers
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

import numpy as np
import pandas as pd

from collections import Counter

import re

import nltk
nltk.download('stopwords')
from nltk import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
stopwords = nltk.corpus.stopwords.words('english')

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#Get device and use GPU if available
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
 
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)   

device=get_default_device()
device

device(type='cuda')

# 1b. Load data from Google Drive and set up preprocesser

Note that the dataset cannot be accessed from HuggingFace directly. It must be downloaded from its Kaggle page.

In [ ]:
# Setting the drive
drive.mount('/content/gdrive')
drive_folder = "gdrive/My Drive/CS6741 - Topics in Natural Language Processing and Machine Learning/CS6741 Replication Project/Final Project"
#drive_folder = "gdrive/My Drive/CS6741 Replication Project/Final Project"

Mounted at /content/gdrive


In [ ]:
# Import data from huggingface
data = datasets.load_dataset('jigsaw_toxicity_pred', data_dir=drive_folder+'/data/jigsaw_toxicity_pred')

Using custom data configuration default-d9843c66a1f7bf76


Dataset jigsaw_toxicity_pred downloaded and prepared to /root/.cache/huggingface/datasets/jigsaw_toxicity_pred/default-d9843c66a1f7bf76/1.1.0/b5a7e4444c940e3254416217128ad87ab7a53c9a54db4c72df349baecd5f43e6. Subsequent calls will reuse this data.


In [ ]:
# Remove all columns in train and test apart from 'comment_text' and 'toxic'
data = data.remove_columns(['severe_toxic', 'obscene', 'threat', 'insult','identity_hate'])

In [ ]:
# Create a function to preprocess the text (which will help find meaningful log-odds differences later on)
# Credit: https://www.kaggle.com/satyamkryadav/bert-model-96-77

punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~"+ '""“”’'+ u"\u25AF"


def clean_special_chars(text, punct):
  for p in punct:
        text=text.lower()
        text = text.replace(p, "")
        text = text.strip('``')
        text = re.sub(r'[0-9]+' , '' ,text)
        text = re.sub(r'&amp;', '&', text)
        text = re.sub(r'\s+', ' ', text).strip()
        encoded_string = text.encode("ascii", "ignore")
        decode_string = encoded_string.decode()
  return decode_string

#Check function output on the previous examples
#train = [clean_special_chars(x, punct) for x in data['train']['comment_text']]
#test = [clean_special_chars(x, punct) for x in data['test']['comment_text']]
#print(train[0:10])

In [ ]:
# Save the true labels of our test set
true_label=data['test']['toxic']
print(len(true_label))

63978


# E0: Run naive model

In [ ]:
# Define the parameters for loading BERT
#model_class, tokenizer_class, pretrained_weights = (transformers.BertForSequenceClassification, transformers.BertTokenizer, 'bert-base-uncased') # To avoid "KeyError: 'loss'" message during train()
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertForSequenceClassification, transformers.DistilBertTokenizerFast, 'distilbert-base-uncased') # To avoid "KeyError: 'loss'" message during train()

# Load tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

In [ ]:
# Preprocess the dataset
def text_preprocessing_byexample(example):
  example['comment_text'] = clean_special_chars(example['comment_text'], punct)
  return example

preprocessed_dataset = data.map(text_preprocessing_byexample)

print('preprocessed_dataset:', preprocessed_dataset)
print('preprocessed_dataset examples:', preprocessed_dataset['train'][0:10]['comment_text'])

def preprocess_function(examples):
  return tokenizer(examples['comment_text'], truncation=True)

encoded_dataset = preprocessed_dataset.map(preprocess_function, batched=True)
encoded_dataset_2 = encoded_dataset.rename_column("toxic", "label") # To avoid "KeyError: 'loss'" message during train()
type(encoded_dataset_2)


preprocessed_dataset: DatasetDict({
    train: Dataset({
        features: ['comment_text', 'toxic'],
        num_rows: 159571
    })
    test: Dataset({
        features: ['comment_text', 'toxic'],
        num_rows: 63978
    })
})
preprocessed_dataset examples: ['explanation why the edits made under my username hardcore metallica fan were reverted they werent vandalisms just closure on some gas after i voted at new york dolls fac and please dont remove the template from the talk page since im retired now', 'daww he matches this background colour im seemingly stuck with thanks talk january utc', 'hey man im really not trying to edit war its just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info', 'more i cant make any real suggestions on improvement i wondered if the section statistics should be later on or a subsection of types of accidents i think the refe

datasets.dataset_dict.DatasetDict

In [ ]:
# Fine-tuning the BERT model

# Load pre-trained BERT model
num_labels = 2 # Toxic labels
model = model_class.from_pretrained(pretrained_weights, num_labels=num_labels)

metric_name = "accuracy"

training_args = TrainingArguments(
    evaluation_strategy = "epoch",   # set the evaluation to be done at the end of each epoch
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                 # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    output_dir='./results',          # output directory
)

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels,preds)
  return {
      'accuracy': acc
  }

trainer = Trainer(
    model,
    training_args,
    train_dataset=encoded_dataset_2["train"],
    eval_dataset=encoded_dataset_2["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier

Epoch,Training Loss,Validation Loss,Accuracy
1,0.093100,0.166074,0.935415
2,0.064500,0.219898,0.927397
3,0.044200,0.320262,0.923614


TrainOutput(global_step=29922, training_loss=0.073197956096679, metrics={'train_runtime': 10639.673, 'train_samples_per_second': 2.812, 'total_flos': 0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 1825714176, 'init_mem_gpu_alloc_delta': 268953088, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 25358336, 'train_mem_gpu_alloc_delta': 815231488, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 6593935872})

In [2]:
#Saving model
cached_model_directory_name = '/results_preprocess'  
trainer.save(drive_folder+cached_model_directory_name)

In [ ]:
#Accuracy results
predicted_results = trainer.predict(encoded_dataset_2["test"])
predicted_results.predictions.shape
predicted_labels = predicted_results.predictions.argmax(-1)
predicted_labels.shape
predicted_labels = predicted_labels.flatten().tolist()
print(classification_report(true_label, predicted_labels))

              precision    recall  f1-score   support

           0       0.98      0.95      0.96     57888
           1       0.62      0.84      0.71      6090

    accuracy                           0.94     63978
   macro avg       0.80      0.89      0.84     63978
weighted avg       0.95      0.94      0.94     63978



## 2. Misclassification Analysis: Examining Misclassified Examples

In [ ]:
#Grab misclassified examples
df=pd.DataFrame(list(zip(true_label, predicted_labels, data['test']['comment_text'])), columns=['true_label', 'predicted_labels', 'comment'])
df.head() 

In [ ]:
#Get false positives and false negatives
df_fp=df[(df['true_label']==0) & (df['predicted_labels']==1)]
df_fn=df[(df['true_label']==1) & (df['predicted_labels']==0)]
df_tp=df[(df['true_label']==1) & (df['predicted_labels']==1)]
df_tn=df[(df['true_label']==0) & (df['predicted_labels']==0)]
print('Misclassified', np.sum(np.where(df['true_label']!=df['predicted_labels'], 1, 0)), 'Correctly classified', np.sum(np.where(df['true_label']==df['predicted_labels'], 1, 0)))

In [ ]:
#Export to results
df_fp.to_csv(drive_folder+'/results/fp.csv')
df_fn.to_csv(drive_folder+'/results/fn.csv')

In [ ]:
#False Positives
df_fp2=df_fp.sample(frac=.005)
for x in df_fp2['true_label']:
  for y in df_fp2['predicted_labels']: 
    if (x != y):
      print('TRUE LABEL:', x)
      print('PREDICTED LABEL:', y)
      print('REVIEW TEXT:', df_fp2['comment'][:100])
      print()

TRUE LABEL: 0
PREDICTED LABEL: 1
REVIEW TEXT: 3393     Is it too much to ask that vandalism, idiotic ...
18016    I read a lot about history, and find that shit...
10512    " \n == This is the worst fucking article I ha...
38160    == Oh fucking come on, do you expect it NOT to...
27114           ::Now it's out of my head (GPL brain dump)
57369    :::::Yup...and thanks to assholes like me and ...
25590    == Retake lies == \n\n This section is full of...
61626    ==Alexis Gounaris== \n Cheeky time wasting los...
39147    01B== \n  IO011990,                01B,      5...
14699    *If you go to his website, after a while an ev...
58122    == is it just my crappy computer? == \n\n or i...
60775                         so this hass stuff is stupid
35827                       GUESS what: yer dad \n\n dildo
30967    " \n\n It's not the greatest attraction, howev...
63025    ::::Maybe you should just take a deep breath a...
Name: comment, dtype: object

TRUE LABEL: 0
PREDICTED LABEL: 1
REVIEW

In [ ]:
#False Negatives
df_fn2=df_fn.sample(frac=.03)
for x in df_fn2['true_label']:
  for y in df_fn2['predicted_labels']: 
    if (x != y):
      print('TRUE LABEL:', x)
      print('PREDICTED LABEL:', y)
      print('REVIEW TEXT:', df_fn2['comment'][:100])
      print()

Streaming output truncated to the last 5000 lines.
20654    == There is NO TURK IN TUNISIA AT ALL AT ALL!!...
19534    Its a good thing GIANDREA that you admit you a...
24957         ==  \n == I like poop! == \n\n ==  == \n  ==
29291    Hello \n Were you sexually abused as a kid? Th...
33386    *I think there should be some mention of how h...
62378    TO all My Pakistani Brothers:- you can not jud...
38999    " \n == bias in the article == \n\n I can't qu...
15791    Pride? This school disgusts me. I hear stories...
20834    " \n\n == The word ""penetration"" == \n\n Goo...
36146    I can't believe this article exists.  This is ...
61839    == Putin == \n\n Your addition of a gay porn i...
55012    " \n\n ==Calvin999== \n You remember the above...
36547    u r krazy Asian woman? I don't think the quest...
40060    *Going out of your way to add the photos to ca...
30059    Deliogul you dont know what you are talking ab...
16749    I have a problem with editing!? Hello incase y...
743  

# 3. Method: Logg-odds for most unique words in test

We begin by preprocessing our text so that we can see what the most unique words are in the test versus train examples (sans stopwords). Note that, apart from the stopwords, the preprocessing function is identical to the one ran beforehand - we replicate so that we are performing logg-odds calculations on the same inputs that BERT sees during fine-tuning. 

We then calculate logg-odds because, in the field, we wouldn't know the "true" labels of incoming examples; however, we can see how different the words of those incoming examples are from the words in our training set through a metric like logg-odds. We simulate that situation here by calculating the logg-odds for the words in our training/test datasets. 

We would 

Log-odds - why aren't we running this using the misclassified examples? This is because 

We notice that our accuracy is really low, so we take 10 examples 

In [ ]:
#Pre-processing
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~"+ '""“”’'+ u"\u25AF"
#stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
stopwords2 = ["padding", "px", "e", "rowspan", "styleverticalalign"]

def clean_special_chars(text, punct):
  for p in punct:
        text=text.lower()
        text = text.replace(p, "")
        text = text.strip('``')
        text = re.sub(r'[0-9]+' , '' ,text)
        text = re.sub(r'&amp;', '&', text)
        text = re.sub(r'\s+', ' ', text).strip()
        encoded_string = text.encode("ascii", "ignore")
        decode_string = encoded_string.decode()
  return decode_string

def preprocess_text(text, punct):
    clean_text=clean_special_chars(text, punct)
    clean_text=re.split('\W+', clean_text)
    clean_text=[token for token in clean_text if token not in stopwords]  
    clean_text=[token for token in clean_text if token not in stopwords2]  
    return " ".join(clean_text)

#Check function output on the previous examples
train = [preprocess_text(x, punct) for x in data['train']['comment_text']]
test = [preprocess_text(x, punct) for x in data['test']['comment_text']]

In [ ]:
print(train[0:10])

['explanation edits made username hardcore metallica fan reverted werent vandalisms closure gas voted new york dolls fac please dont remove template talk page since im retired', 'daww matches background colour im seemingly stuck thanks talk january utc', 'hey man im really trying edit war guy constantly removing relevant information talking edits instead talk page seems care formatting actual info', 'cant make real suggestions improvement wondered section statistics later subsection types accidents think references may need tidying exact format ie date format etc later noone else first preferences formatting style references want please let know appears backlog articles review guess may delay reviewer turns listed relevant form eg wikipediagoodarticlenominationstransport', 'sir hero chance remember page thats', 'congratulations well use tools well talk', 'cocksucker piss around work', 'vandalism matt shirvington article reverted please dont banned', 'sorry word nonsense offensive anywa

In [ ]:
#Create a counter for the training and test datasets separately

train_count=Counter()
test_count=Counter()

#for line in data['train']['comment_text']:
for line in train:
    tokens=word_tokenize(line.strip())
    for token in tokens:
        train_count[token]+=1

#for line in data['test']['comment_text']:
for line in test:
    tokens=word_tokenize(line.strip())
    for token in tokens:
        test_count[token]+=1

In [ ]:
#Getting a counter for ALL vocabulary in train/test

count=Counter()

#for line in data['train']['comment_text']:
for line in train:
    tokens=word_tokenize(line.strip())
    for token in tokens:
        count[token]+=1

#for line in data['test']['comment_text']:
for line in test:
    tokens=word_tokenize(line.strip())
    for token in tokens:
        count[token]+=1

In [ ]:
#Getting the entire vocabulary
vocabulary = list(count.keys())
vocab_train = list(train_count.keys())
vocab_test = list(test_count.keys())
vocab_size = len(vocab_train)+len(vocab_test)
vocab_set = set(vocab_train) | set(vocab_test)
len(set(vocab_train) | set(vocab_test))

298952

In [ ]:
#Calculating word proportions
word_counts = np.zeros((len(vocabulary), 2))

for row, word in enumerate(vocabulary):
    word_counts[row,0] = train_count[word]
    word_counts[row,1] = test_count[word]

sums = word_counts.sum(axis=0)
word_proportions = word_counts / sums[np.newaxis,:]

In [ ]:
#Plotting differences between train and test
def plot_words(y_value):
    y_std = y_value.std()
    pyplot.figure(figsize=(6,18))
    pyplot.scatter(log_type_frequencies, y_value, alpha=0.3)
    for i in range(len(vocabulary)):
        if np.abs(y_value[i]) > 2 * y_std:
            pyplot.text(log_type_frequencies[i], y_value[i], vocabulary[i])
    pyplot.show()

In [ ]:
#Plotting differences between train and test
from matplotlib import pyplot
type_frequencies = word_counts.sum(axis=1)
log_type_frequencies = np.log(type_frequencies)
#plot_words(word_proportions[:,0] - word_proportions[:,1])

In [ ]:
#Smoothing the word counts - some words have 0 freq
smoothed_word_counts = word_counts + 1.0
smoothed_author_sums = smoothed_word_counts.sum(axis=0)
smoothed_word_odds = smoothed_word_counts / (smoothed_author_sums[np.newaxis,:] - smoothed_word_counts)

In [ ]:
#Using smoothed values to log odd difference between test/train
log_odds_difference = np.log(smoothed_word_odds[:,0] / smoothed_word_odds[:,1])
log_odds_variances = 1.0 / smoothed_word_counts[:,0] + 1.0 / smoothed_word_counts[:,1]

In [ ]:
#Sorting the log-odd/vocabulary pair - get top 25
def sort_words(scores, n=25):
    sorted_pairs = sorted(zip(scores, vocabulary))
    sorted_words = [w for s,w in sorted_pairs]
    print("[more frequently in test] ", ", ".join(sorted_words[:n]))
    print("...")
    print(", ".join(sorted_words[-n:]), " [more frequently in train]")
sort_words(log_odds_difference / np.sqrt(log_odds_variances), 25)

[more frequently in test]  poop, dicks, gay, youfuck, niggers, bitch, fuck, wtf, youi, prick, stupid, hole, poo, da, boob, willy, bananas, traitor, die, nigger, penis, curps, und, fags, kill
...
pages, fish, thank, lol, wanker, attack, jew, admin, contributions, deletion, pig, blocked, edit, edits, personal, wiki, utc, moron, please, contribs, block, user, wikipedia, page, talk  [more frequently in train]


In [ ]:
#Finding the most different words (word counts total)
word_odds=log_odds_difference / np.sqrt(log_odds_variances)
word_counts_tot=word_counts[:,0] + word_counts[:,1]
odds=pd.DataFrame(list(zip(word_counts_tot,  word_counts[:,1], word_odds, vocabulary)), columns=['word_counts','word counts test', 'logg_odds', 'vocabulary'])
#odds.to_csv(drive_folder+'/results/odds.csv')
odds=odds.sort_values(by=['logg_odds'])
odds.head()

,word_counts,word counts test,logg_odds,vocabulary
10923,2220.0,1700.0,-41.192611,poop
30329,1620.0,1454.0,-37.258679,dicks
865,5667.0,2936.0,-35.811830,gay
96184,1480.0,1165.0,-34.449425,youfuck
3316,1217.0,1079.0,-32.509025,niggers


In [ ]:
print(sum('poop' in s for s in train), sum('poop' in s for s in test))

93 150


# Experiment 1

In [ ]:
#Using the test dataset to get examples (These aren't misclassified - just the raw test)
test=data['test']['comment_text']
df=pd.DataFrame(test, columns=['comment'])
df['index'] = df.index
df.head()

,comment,index
0,Thank you for understanding. I think very high...,0
1,:Dear god this site is horrible.,1
2,"""::: Somebody will invariably try to add Relig...",2
3,""" \n\n It says it right there that it IS a typ...",3
4,""" \n\n == Before adding a new product to the l...",4


In [ ]:
#Get list of top 20 words that are unique to test 
def sort_words(scores, n=30):
  sorted_pairs = sorted(zip(scores, vocabulary))
  sorted_words = [w for s,w in sorted_pairs]
  final_list=sorted_words[:n]
  return final_list
test_final=sort_words(word_odds)
print(test_final)

['poop', 'dicks', 'gay', 'youfuck', 'niggers', 'bitch', 'fuck', 'wtf', 'youi', 'prick', 'stupid', 'hole', 'poo', 'da', 'boob', 'willy', 'bananas', 'traitor', 'die', 'nigger', 'penis', 'curps', 'und', 'fags', 'kill', 'je', 'middle', 'fucking', 'r', 'fucked']


In [ ]:
#Emerging words - grab first 20 
df.sample(frac=1) #randomly shuffle 
total=0
def check_list(text, elist, emerge_word=0):
    global total
    clean_text=re.split('\W+', text)
    for token in clean_text:
       if (token in elist) and (total<=19):
          emerge_word+=1
       if (token in elist) and (emerge_word==1) and (total<=19):
          elist.remove(token)
          total += 1
    final_count=np.sum(emerge_word)
    return final_count
df['tag'] = df['comment'].apply(lambda x: check_list(x, test_final))
df['tag']=np.where(df['tag']>0.0, 1, 0)

In [ ]:
df.head()

,comment,index,tag
0,Thank you for understanding. I think very high...,0,0
1,:Dear god this site is horrible.,1,0
2,"""::: Somebody will invariably try to add Relig...",2,0
3,""" \n\n It says it right there that it IS a typ...",3,0
4,""" \n\n == Before adding a new product to the l...",4,0


In [ ]:
#Experiment 1 - creating dataframe to add to train AND creating new test that drops old examples 
df_final=df[df['tag']==1] #keeps the 20 examples with the emerging words (1 example per emerging word)
final_index=df_final['index'].to_list() #.pop(0) removes by index, .remove() removes based on the value in the list 
e1_train_add=data['test'].select(final_index)
e1_test=data['test']
for i in final_index:
  e1_test=e1_test.filter(lambda example, indice: (indice!=i) , with_indices=True)
print('train add examples:', len(e1_train_add), 'test drop examples:', len(e1_test))


train add examples: 20 test drop examples: 63978


In [ ]:
#Creating new train dataset
e1_train = concatenate_datasets([data['train'], e1_train_add])
print('Old train size', len(data['train']), 'New train size', len(e1_train))

Old train size 159571 New train size 159591


In [ ]:
# Define the parameters for loading BERT
#model_class, tokenizer_class, pretrained_weights = (transformers.BertForSequenceClassification, transformers.BertTokenizer, 'bert-base-uncased') # To avoid "KeyError: 'loss'" message during train()
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertForSequenceClassification, transformers.DistilBertTokenizerFast, 'distilbert-base-uncased') # To avoid "KeyError: 'loss'" message during train()

# Load tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

In [2]:
# Preprocess the dataset
def text_preprocessing_byexample(example):
  example['comment_text'] = clean_special_chars(example['comment_text'], punct)
  return example

e1_train_prep = e1_train.map(text_preprocessing_byexample)
e1_test_prep = e1_test.map(text_preprocessing_byexample)

def preprocess_function(examples):
  return tokenizer(examples['comment_text'], truncation=True)

e1_train_encoded = e1_train_prep.map(preprocess_function, batched=True)
e1_train_encoded_2 = e1_train_encoded.rename_column("toxic", "label") 

e1_test_encoded = e1_test_prep.map(preprocess_function, batched=True)
e1_test_encoded_2 = e1_test_encoded.rename_column("toxic", "label") 

In [ ]:
# Fine-tuning the BERT model
# Note: Close all other Collab notebooks to ensure memory is sufficient

# Load pre-trained BERT model
num_labels = 2 # Toxic labels
model = model_class.from_pretrained(pretrained_weights, num_labels=num_labels)

metric_name = "accuracy"

training_args = TrainingArguments(
    evaluation_strategy = "epoch",   # set the evaluation to be done at the end of each epoch
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                 # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    output_dir='./results_e1',          # output directory
)

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels,preds)
  return {
      'accuracy': acc
  }

trainer = Trainer(
    model,
    training_args,
    train_dataset=e1_train_encoded_2,
    eval_dataset=e1_test_encoded_2,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
#Save model
cached_model_directory_name = '/results_e1'  
trainer.save_model(drive_folder+cached_model_directory_name)

In [ ]:
#Create new test labels
true_label=e1_test['toxic']
print(len(true_label))

In [ ]:
#Accuracy score 
predicted_results = trainer.predict(e1_test_encoded_2)
predicted_results.predictions.shape
predicted_labels = predicted_results.predictions.argmax(-1)
predicted_labels.shape
predicted_labels = predicted_labels.flatten().tolist()
print(classification_report(true_label, predicted_labels))

# Experiment 1 - Control

Remove the test examples from test, but add 20 control examples to train

In [ ]:
#Using control examples
control = load_dataset( 'csv', data_files=['gdrive/My Drive/CS6741 Replication Project/Final Project/data/control.csv'], encoding='ISO-8859-1')
e1_train_control = concatenate_datasets([data['train'], control['train']])
print('Old train size', len(data['train']), 'New train size', len(e1_train_control))

In [ ]:
# Define the parameters for loading BERT
#model_class, tokenizer_class, pretrained_weights = (transformers.BertForSequenceClassification, transformers.BertTokenizer, 'bert-base-uncased') # To avoid "KeyError: 'loss'" message during train()
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertForSequenceClassification, transformers.DistilBertTokenizerFast, 'distilbert-base-uncased') # To avoid "KeyError: 'loss'" message during train()

# Load tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

In [ ]:
# Preprocess the dataset
def text_preprocessing_byexample(example):
  example['comment_text'] = clean_special_chars(example['comment_text'], punct)
  return example

e1_train_control_prep = e1_train_control.map(text_preprocessing_byexample)

def preprocess_function(examples):
  return tokenizer(examples['comment_text'], truncation=True)

e1_train_control_encoded = e1_train_control_prep.map(preprocess_function, batched=True)
e1_train_control_encoded_2 = e1_train_control_encoded.rename_column("toxic", "label") 

In [ ]:
# Fine-tuning the BERT model
# Note: Close all other Collab notebooks to ensure memory is sufficient

# Load pre-trained BERT model
num_labels = 2 # Toxic labels
model = model_class.from_pretrained(pretrained_weights, num_labels=num_labels)

metric_name = "accuracy"

training_args = TrainingArguments(
    evaluation_strategy = "epoch",   # set the evaluation to be done at the end of each epoch
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                 # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    output_dir='./results_e1_control',          # output directory
)

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels,preds)
  return {
      'accuracy': acc
  }

trainer = Trainer(
    model,
    training_args,
    train_dataset=e1_train_control_encoded_2,
    eval_dataset=e1_test_encoded_2,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
#Save model
cached_model_directory_name = '/results_e1_control'  
trainer.save_model(drive_folder+cached_model_directory_name)

In [ ]:
#Accuracy score 
predicted_results = trainer.predict(e1_test_encoded_2)
predicted_results.predictions.shape
predicted_labels = predicted_results.predictions.argmax(-1)
predicted_labels.shape
predicted_labels = predicted_labels.flatten().tolist()
print(classification_report(true_label, predicted_labels))

# Experiment 2

In [ ]:
#Get list of top 20 words that are unique to test 
def sort_words(scores, n=100):
  sorted_pairs = sorted(zip(scores, vocabulary))
  sorted_words = [w for s,w in sorted_pairs]
  final_list=sorted_words[:n]
  return final_list
test_final=sort_words(word_odds)
print(test_final)

['poop', 'dicks', 'gay', 'youfuck', 'niggers', 'bitch', 'fuck', 'wtf', 'youi', 'prick', 'stupid', 'hole', 'poo', 'da', 'boob', 'willy', 'bananas', 'traitor', 'die', 'nigger', 'penis', 'curps', 'und', 'fags', 'kill', 'je', 'middle', 'fucking', 'r', 'fucked', 'anal', 'boi', 'motherfucker', 'faggot', 'de', 'stylecolor', 'em', 'solid', 'der', 'vandal', 'stylefontsize', 'comedy', 'omfg', 'se', 'wang', 'dictatorship', 'k', 'vandalizer', 'labour', 'gray', 'fuckers', 'scratch', 'orly', 'wheels', 'moon', 'xlarge', 'fdffe', 'stinks', 'width', 'hamish', 'bella', 'crap', 'goverment', 'trevor', 'stylebackgroundcolor', 'babies', 'f', 'height', 'styleborder', 'n', 'su', 'anime', 'kids', 'du', 'aligncenter', 'porn', 'harte', 'bums', 'que', 'en', 'projectors', 'pelican', 'france', 'sack', 'ist', 'rubbish', 'cellspacing', 'scruffy', 'thorpe', 'slut', 'ballz', 'la', 'cellpadding', 'mother', 'styleverticalaligntop', 'anus', 'es', 'den', 'fart', 'te']


In [ ]:
# part of speech tag - keep first 20 most frequent nouns 
# https://stackoverflow.com/questions/15388831/what-are-all-possible-pos-tags-of-nltk

pos=nltk.pos_tag(test_final) #nouns-NN, NNP, NNS
df_noun=pd.DataFrame(pos, columns=['word', 'pos'])
df_noun=df_noun[(df_noun['pos']=="NN") | (df_noun['pos']=="NNP") | (df_noun['pos']=="NNS")]
df_noun=df_noun[:25]
test_final=df_noun['word'].to_list()

In [ ]:
#Emerging words (NOUNS) - grab first 20 
df.sample(frac=1) #randomly shuffle 
total=0
def check_list(text, elist, emerge_word=0):
    global total
    clean_text=re.split('\W+', text)
    for token in clean_text:
       if (token in elist) and (total<=19):
          emerge_word+=1
       if (token in elist) and (emerge_word==1) and (total<=19):
          elist.remove(token)
          total += 1
    final_count=np.sum(emerge_word)
    return final_count
df['tag'] = df['comment'].apply(lambda x: check_list(x, test_final))
df['tag']=np.where(df['tag']>0.0, 1, 0)

In [ ]:
#Experiment 2 - creating dataframe to add to train AND creating new test that drops old examples 

df_final=df[df['tag']==1] #keeps the 20 examples with the emerging words (1 example per emerging word)
final_index=df_final['index'].to_list() #.pop(0) removes by index, .remove() removes based on the value in the list 
e2_train_add=data['test'].select(final_index)
e2_test=data['test']
for i in final_index:
  e2_test=e2_test.filter(lambda example, indice: (indice!=i) , with_indices=True)
print('train add examples:', len(e2_train_add), 'test examples after drop:', len(e2_test))


train add examples: 20 test examples after drop: 63978


In [ ]:
#Creating new train dataset
e2_train = concatenate_datasets([data['train'], e2_train_add])
print('Old train size', len(data['train']), 'New train size', len(e2_train))

Old train size 159571 New train size 159591


In [ ]:
# Define the parameters for loading BERT
#model_class, tokenizer_class, pretrained_weights = (transformers.BertForSequenceClassification, transformers.BertTokenizer, 'bert-base-uncased') # To avoid "KeyError: 'loss'" message during train()
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertForSequenceClassification, transformers.DistilBertTokenizerFast, 'distilbert-base-uncased') # To avoid "KeyError: 'loss'" message during train()

# Load tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

In [ ]:
# Preprocess the dataset
def text_preprocessing_byexample(example):
  example['comment_text'] = clean_special_chars(example['comment_text'], punct)
  return example

e2_train_prep = e2_train.map(text_preprocessing_byexample)
e2_test_prep = e2_test.map(text_preprocessing_byexample)

def preprocess_function(examples):
  return tokenizer(examples['comment_text'], truncation=True)

e2_train_encoded = e2_train_prep.map(preprocess_function, batched=True)
e2_train_encoded_2 = e2_train_encoded.rename_column("toxic", "label") 

e2_test_encoded = e2_test_prep.map(preprocess_function, batched=True)
e2_test_encoded_2 = e2_test_encoded.rename_column("toxic", "label") 

In [ ]:
# Fine-tuning the BERT model
# Note: Close all other Collab notebooks to ensure memory is sufficient

# Load pre-trained BERT model
num_labels = 2 # Toxic labels
model = model_class.from_pretrained(pretrained_weights, num_labels=num_labels)

metric_name = "accuracy"

training_args = TrainingArguments(
    evaluation_strategy = "epoch",   # set the evaluation to be done at the end of each epoch
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                 # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    output_dir='./results_e2',          # output directory
)

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels,preds)
  return {
      'accuracy': acc
  }

trainer = Trainer(
    model,
    training_args,
    train_dataset=e2_train_encoded_2,
    eval_dataset=e2_test_encoded_2,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
#Save model
cached_model_directory_name = '/results_e2'  
trainer.save_model(drive_folder+cached_model_directory_name)

In [ ]:
#Create new test labels
true_label=e2_test['toxic']
print(len(true_label))

In [ ]:
#Accuracy score 
predicted_results = trainer.predict(e2_test_encoded_2)
predicted_results.predictions.shape
predicted_labels = predicted_results.predictions.argmax(-1)
predicted_labels.shape
predicted_labels = predicted_labels.flatten().tolist()
print(classification_report(true_label, predicted_labels))

# Experiment 2 - Control

In [ ]:
# Fine-tuning the BERT model
# Note: Close all other Collab notebooks to ensure memory is sufficient

# Load pre-trained BERT model
num_labels = 2 # Toxic labels
model = model_class.from_pretrained(pretrained_weights, num_labels=num_labels)

metric_name = "accuracy"

training_args = TrainingArguments(
    evaluation_strategy = "epoch",   # set the evaluation to be done at the end of each epoch
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                 # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    output_dir='./results_e2_control',          # output directory
)

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels,preds)
  return {
      'accuracy': acc
  }

trainer = Trainer(
    model,
    training_args,
    train_dataset=e1_train_control_encoded_2,
    eval_dataset=e2_test_encoded_2, # Test is different because different examples are dropped; train is consistent among all controls
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
#Save model
cached_model_directory_name = '/results_e2_control'  
trainer.save_model(drive_folder+cached_model_directory_name)

In [ ]:
#Accuracy score 
predicted_results = trainer.predict(e2_test_encoded_2)
predicted_results.predictions.shape
predicted_labels = predicted_results.predictions.argmax(-1)
predicted_labels.shape
predicted_labels = predicted_labels.flatten().tolist()
print(classification_report(true_label, predicted_labels))

# Experiment 3

In [ ]:
#Import misclassified examples
df_fp_import = pd.read_csv(drive_folder+"/results/fp.csv", encoding='utf-8')
df_fn_import = pd.read_csv(drive_folder+"/results/fn.csv", encoding='utf-8')
df=pd.concat([df_fp_import, df_fn_import], axis=0)
df['comment_clean'] = df['comment'].apply(lambda x: preprocess_text(x, punct))

3948

In [ ]:
#Grab random 20 misclassified examples 
df.sample(frac=1) #randomly shuffle 
df=df[:20] #select 20 obs
df=df.rename(columns={'Unnamed: 0': 'index'})
final_index=df['index'].to_list() #.pop(0) removes by index, .remove() removes based on the value in the list 
e3_train_add=data['test'].select(final_index)
e3_test=data['test']
for i in final_index:
  e3_test=e3_test.filter(lambda example, indice: (indice!=i) , with_indices=True)
print('train add examples:', len(e3_train_add), 'test examples after drop:', len(e3_test))


train add examples: 20 test drop examples: 63978


In [ ]:
#Creating new train dataset
e3_train = concatenate_datasets([data['train'], e3_train_add])
print('Old train size', len(data['train']), 'New train size', len(e3_train))

In [ ]:
# Define the parameters for loading BERT
#model_class, tokenizer_class, pretrained_weights = (transformers.BertForSequenceClassification, transformers.BertTokenizer, 'bert-base-uncased') # To avoid "KeyError: 'loss'" message during train()
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertForSequenceClassification, transformers.DistilBertTokenizerFast, 'distilbert-base-uncased') # To avoid "KeyError: 'loss'" message during train()

# Load tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

In [ ]:
# Preprocess the dataset
def text_preprocessing_byexample(example):
  example['comment_text'] = clean_special_chars(example['comment_text'], punct)
  return example

e3_train_prep = e3_train.map(text_preprocessing_byexample)
e3_test_prep = e3_test.map(text_preprocessing_byexample)

def preprocess_function(examples):
  return tokenizer(examples['comment_text'], truncation=True)

e3_train_encoded = e3_train_prep.map(preprocess_function, batched=True)
e3_train_encoded_2 = e3_train_encoded.rename_column("toxic", "label") 

e3_test_encoded = e3_test_prep.map(preprocess_function, batched=True)
e3_test_encoded_2 = e3_test_encoded.rename_column("toxic", "label") 

In [ ]:
# Fine-tuning the BERT model
# Note: Close all other Collab notebooks to ensure memory is sufficient

# Load pre-trained BERT model
num_labels = 2 # Toxic labels
model = model_class.from_pretrained(pretrained_weights, num_labels=num_labels)

metric_name = "accuracy"

training_args = TrainingArguments(
    evaluation_strategy = "epoch",   # set the evaluation to be done at the end of each epoch
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                 # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    output_dir='./results_e3',          # output directory
)

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels,preds)
  return {
      'accuracy': acc
  }

trainer = Trainer(
    model,
    training_args,
    train_dataset=e3_train_encoded_2,
    eval_dataset=e3_test_encoded_2,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
#Save model
cached_model_directory_name = '/results_e3'  
trainer.save_model(drive_folder+cached_model_directory_name)

In [ ]:
#Create new test labels
true_label=e3_test['toxic']
print(len(true_label))

In [ ]:
#Accuracy score 
predicted_results = trainer.predict(e3_test_encoded_2)
predicted_results.predictions.shape
predicted_labels = predicted_results.predictions.argmax(-1)
predicted_labels.shape
predicted_labels = predicted_labels.flatten().tolist()
print(classification_report(true_label, predicted_labels))

# Experiment 3 - Control

In [ ]:
# Fine-tuning the BERT model
# Note: Close all other Collab notebooks to ensure memory is sufficient

# Load pre-trained BERT model
num_labels = 2 # Toxic labels
model = model_class.from_pretrained(pretrained_weights, num_labels=num_labels)

metric_name = "accuracy"

training_args = TrainingArguments(
    evaluation_strategy = "epoch",   # set the evaluation to be done at the end of each epoch
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                 # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    output_dir='./results_e3_control',          # output directory
)

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels,preds)
  return {
      'accuracy': acc
  }

trainer = Trainer(
    model,
    training_args,
    train_dataset=e1_train_control_encoded_2, #this is the same for all - augment with 20 random features
    eval_dataset=e3_test_encoded_2, #remove the same 20 from test as in E3
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
#Save model
cached_model_directory_name = '/results_e3_control'  
trainer.save_model(drive_folder+cached_model_directory_name)

In [ ]:
#Accuracy score 
predicted_results = trainer.predict(e3_test_encoded_2)
predicted_results.predictions.shape
predicted_labels = predicted_results.predictions.argmax(-1)
predicted_labels.shape
predicted_labels = predicted_labels.flatten().tolist()
print(classification_report(true_label, predicted_labels))

# Experiment 4 

Find index of misclassified examples, remove from test, add to train, rerun model

In [ ]:
#Loop through misclassified examples/ get random example with one of the top 
df_fp_import = pd.read_csv(drive_folder+"/results/fp.csv", encoding='utf-8')
df_fn_import = pd.read_csv(drive_folder+"/results/fn.csv", encoding='utf-8')
df=pd.concat([df_fp_import, df_fn_import], axis=0)
df['comment_clean'] = df['comment'].apply(lambda x: preprocess_text(x, punct))
len(df)

In [ ]:
#Get list of top 20 words that are unique to test 
def sort_words(scores, n=30):
  sorted_pairs = sorted(zip(scores, vocabulary))
  sorted_words = [w for s,w in sorted_pairs]
  final_list=sorted_words[:n]
  return final_list
test_final=sort_words(word_odds)
print(test_final)

['poop', 'dicks', 'gay', 'youfuck', 'niggers', 'bitch', 'fuck', 'wtf', 'youi', 'prick', 'stupid', 'hole', 'poo', 'da', 'boob', 'willy', 'bananas', 'traitor', 'die', 'nigger', 'penis', 'curps', 'und', 'fags', 'kill', 'je', 'middle', 'fucking', 'r', 'fucked']


In [ ]:
#Emerging words - grab first 20 
df.sample(frac=1) #randomly shuffle 
total=0
def check_list(text, elist, emerge_word=0):
    global total
    clean_text=re.split('\W+', text)
    for token in clean_text:
       if (token in elist) and (total<=19):
          emerge_word+=1
       if (token in elist) and (emerge_word==1) and (total<=19):
          elist.remove(token)
          total += 1
    final_count=np.sum(emerge_word)
    return final_count
df['tag'] = df['comment'].apply(lambda x: check_list(x, test_final))
df['tag']=np.where(df['tag']>0.0, 1, 0)

In [ ]:
#Experiment 4 - creating dataframe to add to train AND creating new test that drops old examples 
df_final=df[df['tag']==1] #keeps the 20 examples with the emerging words (1 example per emerging word)
df_final=df_final.rename(columns={'Unnamed: 0': 'index'})
final_index=df_final['index'].to_list() #.pop(0) removes by index, .remove() removes based on the value in the list 
e4_train_add=data['test'].select(final_index)
e4_test=data['test']
for i in final_index:
  e4_test=e4_test.filter(lambda example, indice: (indice!=i) , with_indices=True)
print('train add examples:', len(e4_train_add), 'test examples after drop:', len(e4_test))


train add examples: 20 test drop examples: 63978


In [ ]:
#Creating new train dataset
e4_train = concatenate_datasets([data['train'], e4_train_add])
print('Old train size', len(data['train']), 'New train size', len(e4_train))

Old train size 159571 New train size 159591


In [ ]:
# Define the parameters for loading BERT
#model_class, tokenizer_class, pretrained_weights = (transformers.BertForSequenceClassification, transformers.BertTokenizer, 'bert-base-uncased') # To avoid "KeyError: 'loss'" message during train()
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertForSequenceClassification, transformers.DistilBertTokenizerFast, 'distilbert-base-uncased') # To avoid "KeyError: 'loss'" message during train()

# Load tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

In [ ]:
# Preprocess the dataset
def text_preprocessing_byexample(example):
  example['comment_text'] = clean_special_chars(example['comment_text'], punct)
  return example

e4_train_prep = e4_train.map(text_preprocessing_byexample)
e4_test_prep = e4_test.map(text_preprocessing_byexample)

def preprocess_function(examples):
  return tokenizer(examples['comment_text'], truncation=True)

e4_train_encoded = e4_train_prep.map(preprocess_function, batched=True)
e4_train_encoded_2 = e4_train_encoded.rename_column("toxic", "label") 

e4_test_encoded = e4_test_prep.map(preprocess_function, batched=True)
e4_test_encoded_2 = e4_test_encoded.rename_column("toxic", "label") 

In [1]:
# Fine-tuning the BERT model
# Note: Close all other Collab notebooks to ensure memory is sufficient

# Load pre-trained BERT model
num_labels = 2 # Toxic labels
model = model_class.from_pretrained(pretrained_weights, num_labels=num_labels)

metric_name = "accuracy"

training_args = TrainingArguments(
    evaluation_strategy = "epoch",   # set the evaluation to be done at the end of each epoch
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                 # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    output_dir='./results_e4',          # output directory
)

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels,preds)
  return {
      'accuracy': acc
  }

trainer = Trainer(
    model,
    training_args,
    train_dataset=e4_train_encoded_2,
    eval_dataset=e4_test_encoded_2,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
#Save model
cached_model_directory_name = '/results_e4'  
trainer.save_model(drive_folder+cached_model_directory_name)

In [ ]:
#Create new test labels
true_label=e4_test['toxic']
print(len(true_label))

In [ ]:
#Accuracy score 
predicted_results = trainer.predict(e4_test_encoded_2)
predicted_results.predictions.shape
predicted_labels = predicted_results.predictions.argmax(-1)
predicted_labels.shape
predicted_labels = predicted_labels.flatten().tolist()
print(classification_report(true_label, predicted_labels))

# Experiment 4 - Control

* Remove the 20 examples from test (use the same test as in Experiment 4)
* Supplement the training with 20 non-toxic texts from literature to act as a constant (test the hypothesis, is it the case that adding any examples would increase the accuracy by the same amount as adding examples with the emerging words)

In [ ]:
# Fine-tuning the BERT model
# Note: Close all other Collab notebooks to ensure memory is sufficient

# Load pre-trained BERT model
num_labels = 2 # Toxic labels
model = model_class.from_pretrained(pretrained_weights, num_labels=num_labels)

metric_name = "accuracy"

training_args = TrainingArguments(
    evaluation_strategy = "epoch",   # set the evaluation to be done at the end of each epoch
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                 # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    output_dir='./results_e4_control',          # output directory
)

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels,preds)
  return {
      'accuracy': acc
  }

trainer = Trainer(
    model,
    training_args,
    train_dataset=e1_train_control_encoded_2, #this is the same as in experiment 1
    eval_dataset=e4_test_encoded_2, 
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
#Save model
cached_model_directory_name = '/results_e4_control'  
trainer.save_model(drive_folder+cached_model_directory_name)

In [ ]:
#Accuracy score 
predicted_results = trainer.predict(e4_test_encoded_2)
predicted_results.predictions.shape
predicted_labels = predicted_results.predictions.argmax(-1)
predicted_labels.shape
predicted_labels = predicted_labels.flatten().tolist()
print(classification_report(true_label, predicted_labels))